# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [9]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from contextlib import suppress

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](http://nickmccullum.com/algorithmic-trading-python/sp_500_stocks.csv). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [48]:
#url='https://github.com/prashantmalan/algorthmic_trading/blob/main/sp_500_stocks.csv?raw=true'
stocks = pd.read_csv(r'C:\Users\user\Downloads\sp_500_stocks.csv')
print (stocks)
type(stocks)
stocks['Ticker']


    Ticker
0        A
1      AAL
2      AAP
3     AAPL
4     ABBV
..     ...
496    YUM
497    ZBH
498   ZBRA
499   ZION
500    ZTS

[501 rows x 1 columns]


0         A
1       AAL
2       AAP
3      AAPL
4      ABBV
       ... 
496     YUM
497     ZBH
498    ZBRA
499    ZION
500     ZTS
Name: Ticker, Length: 501, dtype: object

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [43]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [13]:
symbol='AMZN'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 93932900,
 'calculationPrice': 'close',
 'change': 4.22,
 'changePercent': 0.0372,
 'close': 117.71,
 'closeSource': 'loffiica',
 'closeTime': 1669616830885,
 'companyName': 'Amazon.com Inc.',
 'currency': 'USD',
 'delayedPrice': 121.74,
 'delayedPriceTime': 1660403046255,
 'extendedChange': -0.119,
 'extendedChangePercent': -0.001,
 'extendedPrice': 117.616,
 'extendedPriceTime': 1685620808945,
 'high': 118.32,
 'highSource': 'ra1 idy e menlticup5eed',
 'highTime': 1736895903244,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 117.4,
 'iexCloseTime': 1710925206731,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 113.52,
 'iexOpenTime': 1682396881750,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'lastTradeTime': 1694901576717,
 'latestPrice': 121.72,
 'latestSource': 'Close',
 'latestTime': 'June 24, 2022',
 'latestUpdate': 1658519561208,
 'latestVolume': 71506406,
 'low':

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [14]:
data['latestPrice']
data['marketCap']

1208410187210

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [15]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
sp_dataframe = pd.DataFrame(columns = my_columns)
sp_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


In [ ]:
sp_dataframe = sp_dataframe.append(
                                        pd.Series(['AAPL', 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
sp_dataframe

## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [18]:
sp_dataframe = pd.DataFrame(columns = my_columns)
print(stocks)
for symbol in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    print(api_url)
    data = requests.get(api_url).json()
    print(data)
    sp_dataframe = sp_dataframe.append(
                                        pd.Series([symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
    print(sp_dataframe)
    


    Ticker
0        A
1      AAL
2      AAP
3     AAPL
4     ABBV
..     ...
499    YUM
500    ZBH
501   ZBRA
502   ZION
503    ZTS

[504 rows x 1 columns]
https://sandbox.iexapis.com/stable/stock/A/quote?token=Tpk_059b97af715d417d9f49f50b51b1c448
{'avgTotalVolume': 1842822, 'calculationPrice': 'close', 'change': 2.8, 'changePercent': 0.02318, 'close': 124.07, 'closeSource': 'ificlfao', 'closeTime': 1690269357765, 'companyName': 'Agilent Technologies Inc.', 'currency': 'USD', 'delayedPrice': 121.91, 'delayedPriceTime': 1727074405514, 'extendedChange': 0.11, 'extendedChangePercent': 0.00093, 'extendedPrice': 121.9, 'extendedPriceTime': 1702704984444, 'high': 124.07, 'highSource': 'edetcrdla5mni1pie uey  ', 'highTime': 1677505123664, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 122.58, 'iexCloseTime': 1727514611554, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 124.76, 'iexOpenTime': 1725226011395, 'iexRealtimePrice': None, 

KeyboardInterrupt: 

In [ ]:
sp_dataframe

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [49]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [50]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])

sp_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
        batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(batch_api_call_url).json()
        print (data.get(BaseException))

        for symbol in symbol_string.split(','):
            sp_dataframe = sp_dataframe.append(
                                            pd.Series([symbol, 
                                                       data[symbol]['quote']['latestPrice'], 
                                                       data[symbol]['quote']['marketCap'], 
                                                       'N/A'], 
                                                      index = my_columns), 
                                            ignore_index = True)


    
sp_dataframe

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD,GIS
GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILMN,INCY,INF

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,122.39,37541592986,N/A
1,AAL,14.50,9370307515,N/A
2,AAP,188.40,11244814424,N/A
3,AAPL,147.87,2300873878968,N/A
4,ABBV,156.36,278088662697,N/A
...,...,...,...,...
496,YUM,114.78,34004850415,N/A
497,ZBH,111.70,23608669415,N/A
498,ZBRA,315.98,16562587201,N/A
499,ZION,55.80,8140093195,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [51]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:10000000


In [55]:
position_size = float(portfolio_size) / len(sp_dataframe.index)
for i in range(0, len(sp_dataframe['Ticker'])):
    sp_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / sp_dataframe['Price'][i])
sp_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,122.39,37541592986,163
1,AAL,14.50,9370307515,1376
2,AAP,188.40,11244814424,105
3,AAPL,147.87,2300873878968,134
4,ABBV,156.36,278088662697,127
...,...,...,...,...
496,YUM,114.78,34004850415,173
497,ZBH,111.70,23608669415,178
498,ZBRA,315.98,16562587201,63
499,ZION,55.80,8140093195,357


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [58]:
writer = pd.ExcelWriter(r'C:\Users\user\Downloads\recommended_trades.xlsx', engine='xlsxwriter')
sp_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [59]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_format #This applies the format 'string_format' to the column
                    )
```

In [60]:
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
# writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)


This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [61]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [62]:
writer.save()